In [11]:
import gradio as gr
import re
import numpy as np
import pandas as pd
from stemmer import Stemmer
from difflib import SequenceMatcher  # Add this import statement


# Function to process input
def predict_disease(input_text):
    data = pd.read_excel("Gujarati_Dimensionality_Reduction.xlsx")
    output = ""

    s1 = "રોગ"
    if input_text.startswith(s1):
        flag = 1
        stemmer = Stemmer()
        stemmed_text = stemmer.stem(input_text)
        stemmed_words = re.split(r'[;|,|\s]\s*', stemmed_text)

        diseases_dire = []

        for i in range(len(stemmed_words)):
            for col in data['રોગ']:
                if stemmed_words[i] == col and stemmed_words[i] != 'રોગ':
                    diseases_dire.append(stemmed_words[i])

        if len(diseases_dire) > 0:
            output = "રોગો મેળ ખાતા ફોર્મ ડેટાસેટ: " + ", ".join(diseases_dire)
        else:
            output = "કોઈ રોગ મેળ ખાતો નથી."
    else:
        flag = 0
        stemmer = Stemmer()
        input_text = input_text.replace("અને", ",")
        stemmed_text = stemmer.stem(input_text)
        stemmed_words = re.split(r'(;|,|\s)\s*', stemmed_text)

        f = open("guj_pos_tag.txt", mode='r', encoding='UTF-8')
        data = f.read()
        sentences = data.split('\n')[1:]
        words = []
        for s in sentences:
            words.append(s.split('\t')[1])

        for i in range(len(sentences)):
            words[i] = re.sub(r'[^.A-Zઁ-૱\\,_-]', ' ', words[i])
        pairs = []
        for i in range(len(words)):
            pairs.append(words[i].split(" "))
        tagged_guj_sentences = []

        for i in range(len(pairs)):
            for j in range(len(pairs[i])):
                if len(pairs[i][j].split("\\")) == 2:
                    k, v = pairs[i][j].split("\\")
                    tagged_guj_sentences.append((k, v))

        vocab = [word for word, tag in tagged_guj_sentences]
        tags = [tag for word, tag in tagged_guj_sentences]

        stemmer = Stemmer()
        stem_words = []
        for v in vocab:
            stem_words.append(stemmer.stem(v))
        noun_words_intxt = dict(zip(stem_words, tags))

        compared_words = []
        maxn = 0
        for j in range(len(stemmed_words)):
            for key, value in noun_words_intxt.items():
                if value != 'મ':
                    if value == 'N_NN' or value == 'RD_PUNC':
                        if stemmed_words[j] == key:
                            compared_words.append(stemmed_words[j])

        def listToString(s):
            str1 = ""
            for ele in s:
                str1 = str1 + ele + " "
            return str1

        listnoun = listToString(compared_words)
        symptoms_byuser = re.split(r'[,]\s*', listnoun)
        symptoms_byuser = np.array(symptoms_byuser)
        for i in range(len(symptoms_byuser)):
            symptoms_byuser[i] = symptoms_byuser[i].strip()

        symptoms_byuser = symptoms_byuser.tolist()

        symptoms_byuser = np.array(symptoms_byuser)
        data = pd.read_excel("Gujarati_Dimensionality_Reduction.xlsx")
        symptoms = []
        max_n = []
        symptom = 0

        for i in range(len(symptoms_byuser)):
            maxn = 0
            for col in data.columns:
                ratio = SequenceMatcher(None, col, symptoms_byuser[i]).ratio()
                if ratio != 0 and maxn < ratio:
                    maxn = ratio
                    symptom = col
            max_n.append(maxn * 100)
            symptoms.append(symptom)

    output = "\n".join([f"{symptom}\n{maxn:.2f} %" for symptom, maxn in zip(symptoms, max_n)])

    return output

iface1 = gr.Interface(
    fn=predict_disease,
    inputs="text",
    outputs="text",
    live=True,
    title="Gujarati Disease Predictor",
    description="Enter a disease or symptoms in Gujarati to get related information."
)

if __name__ == "__main__":
    iface1.launch()



Running on local URL:  http://127.0.0.1:7941

To create a public link, set `share=True` in `launch()`.


In [12]:
import gradio as gr
import re
import numpy as np
import pandas as pd
from stemmer import Stemmer
from difflib import SequenceMatcher  # Add this import statement

# Function to process input
data = pd.read_excel ("Gujarati_Dimensionality_Reduction.xlsx")

input_text =  input("Enter the disease:") 

s1="રોગ"
if input_text.startswith(s1): 
    flag=1
    stemmer = Stemmer()
    stemmed_text = stemmer.stem(input_text)
    stemmed_words = re.split(r'[;|,|\s]\s*', stemmed_text) 

    diseases_dire=[]

    for i in range(len(stemmed_words)): 
        for col in data['રોગ']: 
            if(stemmed_words[i]==col and stemmed_words[i]!='રોગ'):
                diseases_dire.append(stemmed_words[i])

    print("રોગો મેળ ખાતા ફોર્મ ડેટાસેટ : ",diseases_dire)
    diseases_dire=np.array(diseases_dire)

    data2 = pd.read_excel ("Gujarati_Dataset2.xlsx")

    Y = data2[data2.columns[0]].as_matrix()
    disease = Y.tolist()

    lendisease2=len(disease)

    columnnum=[]
    diseases_dire=np.array(diseases_dire)
    for i in range(lendisease2):
        for j in diseases_dire:
            if j ==  disease[i]:
                columnnum.append(i)

    disease_row=data2.loc[columnnum,:]
    disease_row=disease_row.to_numpy()
    

else:       
    flag=0
    stemmer = Stemmer()
    input_text=input_text.replace("અને", ",")
    stemmed_text = stemmer.stem(input_text)
    stemmed_words = re.split(r'(;|,|\s)\s*', stemmed_text) 
    print()
   
    f = open("guj_pos_tag.txt",mode='r',encoding='UTF-8')
    data = f.read()
    sentences = data.split('\n')[1:]
    words = []
    for s in sentences:
        words.append(s.split('\t')[1])

    import re
    for i in range(len(sentences)):
        words[i] = re.sub(r'[^.A-Zઁ-૱\\,_-]',' ',words[i])
    pairs = []
    for i in range(len(words)):
        pairs.append(words[i].split(" "))
    tagged_guj_sentences = []

    for i in range(len(pairs)):
        for j in range(len(pairs[i])):
            if len(pairs[i][j].split("\\")) ==2:
                k,v = pairs[i][j].split("\\")
                tagged_guj_sentences.append((k,v))

    vocab=[word for word,tag in tagged_guj_sentences]
    tags=[tag for word,tag in tagged_guj_sentences]

    from stemmer import Stemmer
    stemmer = Stemmer()
    stem_words = []
    for v in vocab:
        stem_words.append(stemmer.stem(v))
    noun_words_intxt = dict(zip(stem_words,tags))
    noun_words_intxt = [(k, v) for k, v in noun_words_intxt.items()]  

    from difflib import SequenceMatcher
    compared_words=[]
    maxn=0
    for j in range(len(stemmed_words)):
        for i in range(len(noun_words_intxt)):
            if(noun_words_intxt[i][0]!='મ'):
                if(noun_words_intxt[i][1]=='N_NN' or noun_words_intxt[i][1]=='RD_PUNC' ):
                    #print(noun_words_intxt[i][0],stemmed_words[j])
                    if(stemmed_words[j]==noun_words_intxt[i][0]):
                        #print(noun_words_intxt[i][0],stemmed_words[j])
                        compared_words.append(stemmed_words[j])
    
    def listToString(s):  

        str1 = ""  

        for ele in s:  
            str1 =str1+ele+" "   

        return str1  

    listnoun=listToString(compared_words)
    symptoms_byuser =  re.split(r'[,]\s*', listnoun) 
    symptoms_byuser=np.array(symptoms_byuser)
    for i in range(len(symptoms_byuser)):
        symptoms_byuser[i]=symptoms_byuser[i].strip()
        
    symptoms_byuser=symptoms_byuser.tolist()

    symptoms_byuser=np.array(symptoms_byuser)

    from difflib import SequenceMatcher
    
    data = pd.read_excel ("Gujarati_Dimensionality_Reduction.xlsx")
    symptoms=[]
    max_n=[]
    symptom=0
    

    for i in range(len(symptoms_byuser)): 
        maxn=0
        for col in data.columns: 
            #print(symptoms_byuser[i])
            ratio = SequenceMatcher(None, col, symptoms_byuser[i]).ratio()
            if(ratio!=0 and maxn<ratio):
                maxn=ratio
                symptom=col
        max_n.append(maxn*100)
        symptoms.append(symptom)

    data = pd.read_excel ("Gujarati_Dimensionality_Reduction.xlsx")
    i=0
    a = []

    for col in data.columns:
        a.append(col) 
        i=i+1

    a.pop() 
    l=len(a)
    s = [0] * l 

    import nltk
    symptoms_word = symptoms
    for i in symptoms_word:
        s[a.index(i)]=1

    s=np.array(s)
    s=s.reshape(-1,1)
    s.shape
    s=s.T

    alldiseases = []
    for i in symptoms_word: 
        for k in range (41):
            if data[i][k] == 1:    
                alldiseases.append(data['રોગ'][k])  

    def countfreq(alldiseases):
        freq_diseases = dict()

        for elem in alldiseases: 
            if elem in freq_diseases:
                freq_diseases[elem] += 1
            else:
                freq_diseases[elem] = 1    

        freq_diseases = { key:value for key, value in freq_diseases.items() if value >= len(symptoms_word)}
        return freq_diseases

    freq_diseases = countfreq(alldiseases)   
    freq_diseases = list(freq_diseases.items())
    commondiseases = np.array(freq_diseases)

    data = pd.DataFrame(commondiseases)
    data=data[data.columns[:-1]]
    commondiseases=data.to_numpy()

    print()
    if len(commondiseases) != 0:
        alldiseases=commondiseases
        output_text = alldiseases
    else:
        x = np.array(alldiseases) 
        alldiseases=np.unique(x)
        output_text = alldiseases

    def display_output(તમને_થઈ_શકે_તેવા_રોગની_સંભાવનાઓ):
        return output_text

    iface = gr.Interface(fn=display_output, inputs="text", outputs="text")
    iface.launch()



Running on local URL:  http://127.0.0.1:7942

To create a public link, set `share=True` in `launch()`.


In [13]:

import gradio as gr
data = pd.read_excel ("Gujarati_Dimensionality_Reduction.xlsx")
disease=data.iloc[:, -1]
len_disease=len(disease)

len_alldisease=len(alldiseases)
Number=[]

j=0
for i in range(len_disease):
    if j != len_alldisease:
        if alldiseases[j] ==  disease[i]:
            Number.append(i)
            j=j+1
    i=i+1

feature_row=data.loc[Number,:]
feature_row=feature_row.to_numpy()

features=[]
for j in range(len_alldisease):
    if j !=len_alldisease:
        for i in range(128):
            if feature_row[j][i] == 1:
                features.append(a[i])
            i=i+1

for k in range(len(symptoms_word)):
    j=0
    for j in features:
        if symptoms_word[k] == j:
            features.remove(j);           
    k=k+1   

features=np.unique(features)

if features != []:
    output2 = features
def display_output2(આ_અન્ય_લક્ષણો_છે):
    return output2

iface = gr.Interface(fn=display_output2, inputs="text", outputs="text")
iface.launch()

lst = [ ] 
n = len(features)
s=s.T
output3 = []
for m in features: 
    ele = inputs = (input())
    if ele == "stop":
        break
    if ele != "0":
        s[a.index(m)]=1
    if ele!="1" and ele !="0":
        break
    output3 = output3 + [m]    
def display_output3(પ્રદાન_કરાયેલા_લક્ષણો_છે):
    return output3

iface = gr.Interface(fn=display_output3, inputs="text", outputs="text")
iface.launch()

s=s.T
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data_temp = pd.read_excel ("Gujarati_Training.xlsx")
X = data_temp.iloc[:, :-1].values
Y = data_temp.iloc[:, -1].values
y = labelencoder.fit_transform(Y)

from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X, y)
pr1= classifier.predict(s)
R1=labelencoder.inverse_transform(pr1)

data = pd.read_excel ("Gujarati_Dimensionality_Reduction.xlsx")
disease_original=data.loc[data['રોગ'].isin(R1)]
temp=disease_original.drop(columns=['રોગ'])
disease_original = np.array(temp)
s=np.array(s)

count=0
total=0

for j in range(len(disease_original.T)):
    if (disease_original.T[j]==1):
        total=total+1

for i in range(len(disease_original.T)):
    if (s.T[i]==disease_original.T[i] and disease_original.T[i]==1):
        count=count+1

prp = (count / total) * 100
output = "\n".join([f"{R1}\n{prp:.2f} %"])

def display_output(દાખલ_કરેલા_લક્ષણો_પરથી_રોગ_થવાની_સંભાવના):
    return output

iface = gr.Interface(fn=display_output, inputs="text", outputs="text")
iface.launch()



C:\Users\91996\AppData\Local\Temp\ipykernel_19944\2259719346.py:37: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if features != []:


Running on local URL:  http://127.0.0.1:7943

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7944

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7945

To create a public link, set `share=True` in `launch()`.


In [14]:
import pandas as pd

data2 = pd.read_excel ("Gujarati_Dataset2.xlsx")

Y = data2[data2.columns[0]].values
disease = Y.tolist()

lendisease2=len(disease)

columnnum=[]
j=0
for i in range(lendisease2):
    if R1 ==  disease[i]:
        columnnum.append(i)

disease_row=data2.loc[columnnum,:]
disease_row=disease_row.to_numpy()

Description = 1

output1=(disease_row[0][1])
def display_output(રોગનું_વર્ણન):
    return output1
iface = gr.Interface(fn=display_output, inputs="text", outputs="text")
iface.launch()

output2=(disease_row[0][2])
def display_output2(રોગ_નિવારણ_પદ્ધતિઓ):
    return output2
iface = gr.Interface(fn=display_output2, inputs="text", outputs="text")
iface.launch()

output3=(disease_row[0][3])
def display_output3(ઘરગથ્થુ_ઉપચાર):
    return output3
iface = gr.Interface(fn=display_output3, inputs="text", outputs="text")
iface.launch()

output4 = (disease_row[0][4])
def display_output4(રોગનૂ_ઉપચાર):
    return output4
iface = gr.Interface(fn=display_output4, inputs="text", outputs="text")
iface.launch()


Running on local URL:  http://127.0.0.1:7946

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7947

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7948

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7949

To create a public link, set `share=True` in `launch()`.
